# MCP + AI Agent Interaction (Beginner Demo)

This notebook shows **the same AI agent** in two modes:

1) **Without MCP (the pain):** the agent is tightly coupled to specific Python functions/tools.  
2) **With MCP (the fix):** tools are exposed via a **Model Context Protocol (MCP) server**, and the agent becomes a **generic client** that can discover and use tools consistently.

> ✅ No web-scraping / BeautifulSoup.  
> ✅ Uses **Tavily** for web search (snippets + URLs).  
> ✅ Uses **OpenAI** for summarization (via environment variables).  
> ✅ Ends with a compact summary **inside the notebook**.

**Date in this demo:** 2026-01-08


## 0) Install dependencies (single cell)

If you re-run the notebook, you can re-run this cell safely.


In [ ]:
# If you face version conflicts, restart the runtime and run this cell again.
!pip -q install -U openai tavily-python "mcp[cli]" nest_asyncio
import os, json, textwrap, datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 5.9 MB/s eta 0:00:00


## 1) Set API keys (environment variables only)

We will use:
- `OPENAI_API_KEY`
- `OPENAI_BASE_URL` (optional, default: https://api.openai.com/v1)
- `TAVILY_API_KEY`


In [ ]:
# Cell 2 — Configure API keys (safe prompting)
import os, getpass
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

OPENAI_BASE_URL = "https://aibe.mygreatlearning.com/openai/v1"

os.environ["OPENAI_API_KEY"] = openai_api_key

os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

## 2) Common helper: a tiny “audit-safe” summarizer

We keep the LLM usage simple:
- The model **must** include a disclaimer
- The model **must** cite sources as `[1] [2] ...`
- If sources are weak, it must say **insufficient authoritative guidance**


In [ ]:
from openai import OpenAI

def now_utc_iso():
    return datetime.datetime.utcnow().replace(microsecond=0).isoformat() + "Z"

def make_openai_client():
    # OpenAI Python SDK supports base_url for proxies/gateways.
    return OpenAI(
        api_key=openai_api_key,
        base_url=OPENAI_BASE_URL
    )

def audit_safe_summarize(user_query: str, evidence: dict, strictness: str = "conservative") -> str:
    """Given evidence (search snippets + urls), produce a structured audit-style response."""
    client = make_openai_client()
    verified = now_utc_iso()

    instructions = f"""You are an Audit & Compliance research assistant.
Rules (non-negotiable):
- This is NOT legal or tax advice.
- Do NOT provide filing instructions or tax avoidance strategies.
- Summarize ONLY what the evidence supports. No speculation.
- If evidence is not authoritative or insufficient, explicitly say: "Insufficient authoritative guidance found."
- Use the output structure:
  A) Applicable Sources (title, authority if known, date if known, URL)
  B) Key Points (what the sources say)
  C) Risks / Compliance Notes (based strictly on sources)
  D) Assumptions & Limits
  E) Citations (numbered)
- Include: Last verified on {verified}
Strictness: {strictness}
"""

    # We use the Responses API (recommended) and keep inputs minimal.
    response = client.responses.create(
        model="gpt-4o-mini",
        instructions=instructions,
        input=f"User query: {user_query}\n\nEvidence JSON:\n{json.dumps(evidence, ensure_ascii=False)[:12000]}"
    )
    return response.output_text


# Part A — Without MCP (the challenge)

### What we do here
We build a simple agent that can:
- call Tavily search
- call a local “policy” function

### The problem
Every new tool requires:
- editing the agent code
- re-registering schemas
- custom glue code per integration

This becomes an **N×M integration mess** as tools and apps grow.


In [ ]:
from tavily import TavilyClient

tavily = TavilyClient(api_key=userdata.get('TAVILY_API_KEY'))

# Tool 1 (direct): search
def search_web_direct(query: str, max_results: int = 5) -> dict:
    res = tavily.search(query=query, max_results=max_results, include_answer=False)
    # We keep only lightweight evidence: title + url + snippet
    cleaned = []
    for item in res.get("results", []):
        cleaned.append({
            "title": item.get("title"),
            "url": item.get("url"),
            "content": item.get("content")
        })
    return {"results": cleaned}

# Tool 2 (direct): internal policy (pretend this came from a policy repository)
def internal_policy_direct() -> str:
    return textwrap.dedent("""    Responsible AI Guardrails (Demo Policy)
    - Not legal/tax advice; provide research summaries only.
    - Prefer official tax authority / government sources.
    - Avoid speculative interpretation; quote/reflect source language.
    - If low confidence or sources are weak: say insufficient authoritative guidance.
    - Never provide filing steps, tax planning, or avoidance techniques.
    """).strip()

def agent_without_mcp(user_query: str, max_sources: int = 5, strictness: str = "conservative") -> str:
    # Tight coupling: the agent knows exactly which functions exist and how to call them.
    policy = internal_policy_direct()
    evidence = search_web_direct(user_query, max_results=max_sources)

    packed = {
        "policy": policy,
        "search": evidence,
    }
    return audit_safe_summarize(user_query=user_query, evidence=packed, strictness=strictness)

# Quick smoke test
print(agent_without_mcp("UAE VAT penalty for late filing", max_sources=3)[:700])


/tmp/ipython-input-1149917053.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.datetime.utcnow().replace(microsecond=0).isoformat() + "Z"


A) Applicable Sources
1. Title: Understanding VAT Late Payment Penalties in UAE  
   Authority: Alaan  
   URL: [alaan.com](https://www.alaan.com/blog/vat-late-payment-penalties-uae)  
2. Title: VAT Late Payment Penalty UAE  
   Authority: Bestax Chartered Accountants  
   URL: [bestaxca.com](https://bestaxca.com/vat-late-payment-penalty-uae-avoid-penalties-now/)  
3. Title: Penalty for Late Payment of VAT in UAE: Complete Guide for 2025  
   Authority: Flying Colour Tax  
   URL: [flyingcolourtax.com](https://www.flyingcolourtax.com/blog/uae-vat-penalty-complete-guide/)  

B) Key Points
- For the first offence of late VAT filing, a penalty of AED 1,000 is incurred. If the violation is repea


## Why “without MCP” is painful (in one paragraph)

Without MCP, your agent is **hard-wired** to custom Python functions: every time you add a new capability (e.g., “read internal circulars”, “query a database”, “call a calculator”), you must manually write glue code, keep tool schemas consistent, and update the agent’s routing logic. Across many apps and many tools, this creates an **N×M connector problem** (many tools × many clients), which slows development, increases bugs, and makes governance/permissions harder to standardize.


# Part B — With MCP (the resolution)

Now we expose tools through an **MCP server** and connect with a standard **MCP client**.

### What changes?
- Tools live behind an MCP server (a reusable “tool box”).
- The agent becomes a generic client that can:
  - discover available tools
  - call them by name with a consistent protocol

So you can swap / add tools with **minimal client changes**.


## 3) Create a tiny MCP server (two tools)

The server exposes:
- `search_tax_guidance(query, max_results)`  → Tavily search snippets + URLs  
- `get_responsible_ai_policy()`              → returns a policy string

We write it to a `.py` file so the MCP client can launch it via **stdio**.


In [ ]:
server_code = r'''
import os, textwrap
from tavily import TavilyClient
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Tax-MCP-Tools")

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

@mcp.tool()
def search_tax_guidance(query: str, max_results: int = 5) -> dict:
    """Search tax laws / official guidance on the web (returns snippets + URLs)."""
    res = tavily.search(query=query, max_results=max_results, include_answer=False)
    cleaned = []
    for item in res.get("results", []):
        cleaned.append({
            "title": item.get("title"),
            "url": item.get("url"),
            "content": item.get("content"),
        })
    return {"results": cleaned}

@mcp.tool()
def get_responsible_ai_policy() -> str:
    """Return the Responsible AI policy text that the agent must follow."""
    return textwrap.dedent("""    Responsible AI Guardrails (Demo Policy)
    - NOT legal/tax advice. Provide research summaries only.
    - Prefer official tax authority / government sources.
    - No speculation; summarize only what sources state.
    - If low confidence: "Insufficient authoritative guidance found."
    - No filing instructions, tax planning, or avoidance techniques.
    - Always include retrieval timestamp ("Last verified on ...").
    """).strip()

def main():
    # Default transport is STDIO.
    mcp.run()

if __name__ == "__main__":
    main()
'''
open("tax_mcp_server.py", "w", encoding="utf-8").write(server_code)
print("✅ Wrote tax_mcp_server.py")


✅ Wrote tax_mcp_server.py


## 4) MCP client: connect, discover tools, call tools

We will:
1) Start a stdio session to the server script  
2) List available tools  
3) Call tools by name (no custom glue per tool)


In [ ]:
# Start MCP server on localhost:8765 (background)
!nohup python tax_mcp_server.py --port 8765 --host 127.0.0.1 > mcp_server.log 2>&1 &

# Quick check: show last server logs
!tail -n 30 mcp_server.log


In [ ]:
import nest_asyncio, asyncio, os
nest_asyncio.apply()

from mcp import ClientSession
from mcp.client.sse import sse_client

# IMPORTANT: do not use stdio_client in Colab
async def mcp_connect_and_list_tools_sse(url: str):
    async with sse_client(url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await session.list_tools()
            return tools

# In notebooks, prefer loop.run_until_complete
loop = asyncio.get_event_loop()
tools = loop.run_until_complete(mcp_connect_and_list_tools_sse("http://127.0.0.1:8765/sse"))

print("✅ MCP tools discovered:")
for t in tools.tools:
    print(" -", t.name)
tools = asyncio.run(mcp_connect_and_list_tools("tax_mcp_server.py"))

ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)

## 5) Agent using MCP tools (generic)

Notice the difference:
- The agent **does not import Tavily directly**
- The agent **does not directly call local tool functions**
- It uses MCP discovery + calls

That’s the key MCP idea: **standard tool connectivity**. citeturn0search0turn3view0turn3view1


In [ ]:
async def call_mcp_tool(session: ClientSession, tool_name: str, tool_args: dict):
    """Generic tool call helper (works for any tool on any MCP server)."""
    return await session.call_tool(tool_name, tool_args)

async def agent_with_mcp(user_query: str, max_sources: int = 5, strictness: str = "conservative") -> str:
    server_params = StdioServerParameters(
        command="python",
        args=["tax_mcp_server.py"],
        env=dict(os.environ),
    )

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # Discover tools (the agent is not hardcoded to tool implementations)
            tools = await session.list_tools()
            available = [t.name for t in tools.tools]

            # Minimal routing for teaching: if tool exists, call it.
            # (In production, the LLM can decide tool usage; here we keep it beginner-simple.)
            evidence = {}
            if "get_responsible_ai_policy" in available:
                policy = await call_mcp_tool(session, "get_responsible_ai_policy", {})
                evidence["policy"] = policy.content[0].text if hasattr(policy, "content") else str(policy)

            if "search_tax_guidance" in available:
                search = await call_mcp_tool(session, "search_tax_guidance", {"query": user_query, "max_results": max_sources})
                # MCP returns typed content; we normalize to JSON-ish dict for the summarizer
                try:
                    txt = search.content[0].text
                    evidence["search"] = json.loads(txt)
                except Exception:
                    evidence["search"] = {"raw": str(search)}

            return audit_safe_summarize(user_query=user_query, evidence=evidence, strictness=strictness)

# Smoke test
print(asyncio.run(agent_with_mcp("UAE VAT penalty for late filing", max_sources=3))[:800])


# Final summary (inside the notebook)

### Without MCP
- The agent is tightly coupled to specific tool functions.
- Adding/removing tools forces code changes in the agent.
- Harder to standardize governance and permissions across many tools.

### With MCP
- Tools live behind an MCP server and are exposed in a standard way (tools/resources/prompts). citeturn3view1
- The agent becomes a reusable MCP client:
  - discovers tools
  - calls tools by name with structured arguments
- This reduces the “N×M connector problem” and makes tool integration cleaner and more scalable. citeturn0search1turn0search5

### What learners should remember
> MCP is like a standardized “port” for AI apps to plug into tools and context, so agents can work with many tools without bespoke integrations. citeturn0search0turn2search3
